In [1]:
import requests
import warnings
from tqdm import tqdm, trange
import pandas as pd 
import streamlit as st 
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
import plotly_express  as px
import plotly.graph_objects as go
import random
import pickle
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [2]:
def get_stock_data(tickerI):
    tickerL = tickerI
    
    
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{tickerL}?period=annual&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    annual_is_df = pd.DataFrame(response.json())
    if annual_is_df.shape == (0, 0):
        annual_is_df =  pd.DataFrame((float('Nan'), float('Nan')))
        last_date_annual = float('Nan')
    else:
        last_date_annual = annual_is_df.loc[0,'date']
        annual_is_df = annual_is_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        annual_is_df = annual_is_df.add_suffix(f'(FY)')
        
    
    
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{tickerL}?period=quarterly&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    quarterly_is_df = pd.DataFrame(response.json())
    if quarterly_is_df.shape == (0, 0):
        quarterly_is_df =  pd.DataFrame((float('Nan'), float('Nan')))
        last_date_quarterly = float('Nan')
    else:
        last_date_quarterly = quarterly_is_df.loc[0,'date']
        quarterly_is_df = quarterly_is_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        quarterly_is_df = quarterly_is_df.add_suffix(f'(FQ)')
        
    
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{tickerL}?period=annual&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    annual_fp_df = pd.DataFrame(response.json())
    if annual_fp_df.shape == (0, 0):
        annual_fp_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        annual_fp_df = annual_fp_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        annual_fp_df = annual_fp_df.add_suffix(f'(FY)')
    
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{tickerL}?period=quarterly&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    quarterly_fp_df = pd.DataFrame(response.json())
    if quarterly_fp_df.shape == (0, 0):
        quarterly_fp_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        quarterly_fp_df = quarterly_fp_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        quarterly_fp_df = quarterly_fp_df.add_suffix(f'(FQ)')
    
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{tickerL}?period=annual&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    annual_cf_df = pd.DataFrame(response.json())
    if annual_cf_df.shape == (0, 0):
        annual_cf_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        annual_cf_df = annual_cf_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        annual_cf_df = annual_cf_df.add_suffix(f'(FY)')
    
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{tickerL}?period=quarterly&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    quarterly_cf_df = pd.DataFrame(response.json())
    if quarterly_cf_df.shape == (0, 0):
        quarterly_cf_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        quarterly_cf_df = quarterly_cf_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        quarterly_cf_df = quarterly_cf_df.add_suffix(f'(FQ)')
    
    stock_data = pd.concat([annual_is_df, quarterly_is_df, annual_fp_df, quarterly_fp_df, annual_cf_df  ], axis = 1)
    stock_data = pd.concat([stock_data,quarterly_cf_df], axis = 1)
    stock_data = stock_data[stock_data.columns.drop(list(stock_data.filter(regex='calendarYear')))]
    stock_data = stock_data[stock_data.columns.drop(list(stock_data.filter(regex='period')))]
    stock_data = stock_data.loc[:,~stock_data.columns.duplicated()].copy()
    stock_data1 = stock_data.iloc[0:9,]
    
    if quarterly_cf_df.shape[0]<9:
        for year in range(1,quarterly_cf_df.shape[0]):
            stock_data1 = pd.concat([stock_data1, pd.DataFrame(stock_data1.iloc[year,0:198].add_suffix(f'-{year}')).transpose().reset_index(drop=True)], axis=1)
    else:
        for year in range(1,9):
            stock_data1 = pd.concat([stock_data1, pd.DataFrame(stock_data1.iloc[year,0:198].add_suffix(f'-{year}')).transpose().reset_index(drop=True)], axis=1)
    stock_data1 = stock_data1.iloc[0,:]
    stock_data_final = pd.DataFrame(stock_data1).transpose()
    stock_data_final.insert(loc = 0, column ="Ticker", value=f"{tickerL}" )
    stock_data_final.insert(loc = 0, column ="Date(FY)", value=f"{last_date_annual}" )
    stock_data_final.insert(loc = 0, column ="Date(FQ)", value=f"{last_date_quarterly}" )
    
    return stock_data_final

In [3]:
us_stock_database = pd.DataFrame()
stock_list_pd = pd.read_pickle("StockList")
stock_list_pd = stock_list_pd.query("type == 'stock'")
stock_list_pd = stock_list_pd.query("exchangeShortName == 'NYSE' | exchangeShortName == 'NASDAQ'")
stock_list_pd.reset_index(inplace=True, drop=True)
stock_list = stock_list_pd["symbol"].to_list()
for ticker in tqdm(stock_list):
    #tqdm._instances.clear()
    us_stock_database = pd.concat([us_stock_database, get_stock_data(ticker)])
    #print(f'processed {ticker}')
    us_stock_database.reset_index(drop=True)


100%|██████████| 10126/10126 [21:10:38<00:00,  7.53s/it]  


In [4]:
us_stock_database

,Date(FQ),Date(FY),Ticker,revenue(FY),costOfRevenue(FY),grossProfit(FY),grossProfitRatio(FY),researchAndDevelopmentExpenses(FY),generalAndAdministrativeExpenses(FY),sellingAndMarketingExpenses(FY),...,ebitdaratio(FY)-1-8,operatingIncome(FY)-1-8,operatingIncomeRatio(FY)-1-8,totalOtherIncomeExpensesNet(FY)-1-8,incomeBeforeTax(FY)-1-8,incomeBeforeTaxRatio(FY)-1-8,incomeTaxExpense(FY)-1-8,netIncome(FY)-1-8,netIncomeRatio(FY)-1-8,eps(FY)-1-8
0,2023-12-31,2023-03-31,NGL-PB,8694904000.0,7923645000.0,771259000.0,0.088702,0.0,71818000.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2023-12-31,2023-12-31,NLY-PI,-1651591000.0,24806000.0,-1676397000.0,1.015019,0.0,42961000.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2023-12-31,2023-12-31,CIO-PA,179096000.0,69997000.0,109099000.0,0.609165,0.0,14841000.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2024-02-25,2023-11-26,LEVI,6179000000.0,2663300000.0,3515700000.0,0.568976,0.0,1267600000.0,1803300000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2023-12-31,2023-12-31,ICL,7654433692.0,5074515754.0,2579917938.0,0.337049,72115816.0,436000000.0,917000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2023-06-30,2022-12-31,UNVR,11475300000.0,8704100000.0,2771200000.0,0.241493,0.0,1282800000.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2023-03-31,2022-12-31,GOGN-WT,0.0,0.0,0.0,0.0,0.0,5397402.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2023-12-31,2023-03-31,VHNAW,13560498.0,5413686.0,8146812.0,0.600775,2670333.0,4648242.0,10736173.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,nan,nan,XPAXW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
